<a href="https://colab.research.google.com/github/Manpreetkour95/NLP/blob/main/Huggingface_trainer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#pip install transformers, datasets
import pandas as pd

# Sample data
data = {
    'text': ['This is a positive review.', "I didn't like this product.", 'The weather is great today.', 'The movie was amazing!', 'I feel sad right now.', 'The food at the restaurant was delicious.', 'The service was terrible.', 'I love my new phone!', 'This book is boring.', 'The trip was fantastic.', "I can't believe I won!", 'The traffic is horrible.', 'The concert was awesome.', "I'm really happy today.", 'The hotel room was clean and comfortable.', 'The test was difficult.', 'The customer service was excellent.', 'I hate waiting in line.', 'The game was fun to play.', "I'm so excited about the party tonight!", "I can't believe I won!", 'The traffic is horrible.', 'The concert was awesome.', "I'm really happy today.", 'The hotel room was clean and comfortable.', 'The test was difficult.', 'The customer service was excellent.', 'I hate waiting in line.', 'The game was fun to play.', "I'm so excited about the party tonight!I can't believe I won!", 'The traffic is horrible.', 'The concert was awesome.', "I'm really happy today.", 'The hotel room was clean and comfortable.', 'The test was difficult.', 'The customer service was excellent.', 'I hate waiting in line.', 'The game was fun to play.', "I'm so excited about the party tonight!", "I can't believe I won!", 'The traffic is horrible.', 'The concert was awesome.', "I'm really happy today.", 'The hotel room was clean and comfortable.', 'The test was difficult.', 'The customer service was excellent.', 'I hate waiting in line.', 'The game was fun to play.', "I'm so excited about the party tonight!", "I'm so excited"],

    'label': ['positive', 'negative', 'positive', 'positive', 'negative', 'positive', 'negative', 'neutral', 'negative', 'positive', 'positive', 'negative', 'positive', 'positive', 'neutral', 'negative', 'positive', 'negative', 'neutral', 'positive', 'positive', 'negative', 'positive', 'positive', 'positive', 'negative', 'positive', 'negative', 'positive', 'positive', 'positive', 'positive', 'negative', 'positive', 'negative', 'positive', 'positive', 'positive', 'negative', 'positive', 'positive', 'positive', 'negative', 'positive', 'positive', 'positive', 'negative', 'positive', 'negative', 'positive']

}
# Create DataFrame
df = pd.DataFrame(data)
unique_labels = df['label'].unique()

# Create a dictionary to map each label to a unique ID
label2id = {label: idx for idx, label in enumerate(unique_labels)}
df['label']=[label2id[label] for label in df.label]
from sklearn.model_selection import train_test_split

train_df, temp_df = train_test_split(df, test_size=0.3, random_state=42)

# Second split: Split the remaining data into validation and test sets
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)
#np.save("train_ids.npy", np.array(train_df.id))
#np.save("test_ids.npy", np.array(test_df.id))
#np.save("tval_ids.npy", np.array(val_df.id))



import os
import warnings
import re
import argparse
import pandas as pd
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments, AdamW
from datasets import load_dataset, Dataset, DatasetDict
warnings.filterwarnings('ignore')

#accuracy=evaluate.load('accuracy')
def prepare_dataset(df):
  #df['text']=df['text']
  df['label']=df['label'].replace(label2id.keys(),label2id.values())

  dataset=DatasetDict()
  dataset['train']=Dataset.from_pandas(train_df[['text','label']].reset_index(drop=True))

  dataset['val']=Dataset.from_pandas(val_df[['text','label']].reset_index(drop=True))
  dataset['test']=Dataset.from_pandas(test_df[['text','label']].reset_index(drop=True))
  return dataset


# Tokenize the data
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

#model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=len(label2id),id2label={y:x for x,y in label2id.items()},label2id=label2id)
model_name = "distilbert-base-uncased"

model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=len(label2id), id2label={y:x for x,y in label2id.items()}, label2id=label2id)

tokenizer = AutoTokenizer.from_pretrained(model_name)
dataset=prepare_dataset(df)
tokenized_datasets = dataset.map(tokenize_function, batched=True)

# Define the TrainingArguments and Trainer
training_args = TrainingArguments(
    output_dir='models/'+model_name,
    learning_rate=2e-5,
    save_strategy='epoch',
    evaluation_strategy="epoch",
    num_train_epochs=4,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=16,
    logging_dir="./logs",
    logging_steps=500)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
)


# Train the model
trainer.train()

trainer.save_model('models/'+model_name)

#os.makedirs(os.path.join('numpy_dump',model_name), exit_ok=True)

